# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770940080076                   -0.53    9.0         
  2   -2.772146381966       -2.92       -1.32    1.0    183ms
  3   -2.772170319869       -4.62       -2.51    1.0    145ms
  4   -2.772170661708       -6.47       -3.30    1.0    146ms
  5   -2.772170721467       -7.22       -3.89    2.0    192ms
  6   -2.772170722979       -8.82       -5.16    1.0    151ms
  7   -2.772170723014      -10.45       -5.51    2.0    172ms
  8   -2.772170723015      -12.10       -6.70    1.0    175ms
  9   -2.772170723015      -13.35       -6.84    2.0    178ms
 10   -2.772170723015   +  -14.40       -7.38    1.0    168ms
 11   -2.772170723015      -14.10       -8.87    2.0    212ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770495219031                   -0.53    9.0         
  2   -2.7

1.7735580630786767

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770733273959                   -0.53    9.0         
  2   -2.772052649750       -2.88       -1.31    1.0    235ms
  3   -2.772082864501       -4.52       -2.57    1.0    145ms
  4   -2.772083409533       -6.26       -3.66    2.0    168ms
  5   -2.772083417188       -8.12       -4.10    2.0    223ms
  6   -2.772083417792       -9.22       -5.23    1.0    150ms
  7   -2.772083417811      -10.74       -6.09    2.0    173ms
  8   -2.772083417811      -13.31       -6.53    1.0    156ms
  9   -2.772083417811      -13.48       -7.20    2.0    213ms
 10   -2.772083417811      -14.18       -7.27    1.0    167ms
 11   -2.772083417811   +  -14.75       -8.35    1.0    160ms

Polarizability via ForwardDiff:       1.7725349597178677
Polarizability via finite difference: 1.7735580630786767
